In [ ]:
import implicit
from scipy.sparse import coo_matrix
import numpy as np
import json
from tqdm import tqdm

In [ ]:
root = "datasets3/"
train_interactions = pd.read_csv(root+"seen_interactions.csv")
test_interactions = pd.read_csv(root+"hidden_interactions.csv")

In [ ]:
# Train BPR model
print("\nTraining BPR model...")
model_bpr = implicit.bpr.BayesianPersonalizedRanking(
    factors=100,
    iterations=50,
    learning_rate=0.01,
    regularization=0.01,
    random_state=42,
    verify_negative_samples=True
)

# BPR expects item-user matrix (transposed)
model_bpr.fit(user_item_matrix.T, show_progress=True)


In [10]:
import implicit
from scipy.sparse import coo_matrix
import numpy as np
import json
from tqdm import tqdm

# Prepare data for BPR

root = "datasets3/"
train_interactions = pd.read_csv(root+"seen_interactions.csv")
test_interactions = pd.read_csv(root+"hidden_interactions.csv")

# Use ALL training data, but weight high ratings more
# This ensures all users are in the model

print(f"Total training interactions: {len(train_interactions)}")

# Create user and item mappings using ALL training data
user_ids = train_interactions['u'].unique()
item_ids = train_interactions['i'].unique()

user_to_idx = {uid: idx for idx, uid in enumerate(user_ids)}
item_to_idx = {iid: idx for idx, iid in enumerate(item_ids)}
idx_to_item = {idx: iid for iid, idx in item_to_idx.items()}
idx_to_user = {idx: uid for uid, idx in user_to_idx.items()}

print(f"Number of users: {len(user_ids)}")
print(f"Number of items: {len(item_ids)}")

# Check for test users not in training
test_users = test_interactions['u'].unique()
train_users = set(user_ids)
users_not_in_train = [u for u in test_users if u not in train_users]
print(f"Test users not in training: {len(users_not_in_train)} out of {len(test_users)}")

# Create user-item interaction matrix with weighted ratings
# Give more weight to higher ratings
user_indices = train_interactions['u'].map(user_to_idx).values
item_indices = train_interactions['i'].map(item_to_idx).values

# Weight strategy: convert ratings to confidence scores
# 5 -> 2.0, 4 -> 1.0, 3 -> 0.5, 2 -> 0.1, 1 -> 0.0 (or exclude)
def rating_to_weight(rating):
    if rating >= 5:
        return 2.0
    elif rating >= 4:
        return 1.0
    elif rating >= 3:
        return 0.5
    elif rating >= 2:
        return 0.1
    else:
        return 0.0

weights = train_interactions['rating'].apply(rating_to_weight).values

# Filter out zero weights
non_zero_mask = weights > 0
user_indices = user_indices[non_zero_mask]
item_indices = item_indices[non_zero_mask]
weights = weights[non_zero_mask]

user_item_matrix = coo_matrix(
    (weights, (user_indices, item_indices)),
    shape=(len(user_ids), len(item_ids))
).tocsr()

print(f"Sparse matrix shape: {user_item_matrix.shape}")
print(f"Number of interactions: {user_item_matrix.nnz}")
print(f"Sparsity: {1 - (user_item_matrix.nnz / (user_item_matrix.shape[0] * user_item_matrix.shape[1])):.4f}")


print("BPR model trained!")

# Build set of items each user has seen
users_seen_train = train_interactions.groupby('u')['i'].agg(set).to_dict()

# Popularity fallback
K = 30
recipe_rating_sum = train_interactions.groupby('i')['rating'].sum()
recipe_rating_count = train_interactions.groupby('i')['rating'].count()
recipe_avg_rating = recipe_rating_sum / recipe_rating_count
recipe_popularity = recipe_avg_rating * recipe_rating_count
top_popular_recipes = list(recipe_popularity.sort_values(ascending=False).index[:K])

# Generate recommendations for test users
recommendations_bpr = {}

for user_id in tqdm(test_users, desc="BPR recommendations"):
    # Check if user exists in training data
    if user_id not in user_to_idx:
        # Unknown user: use popularity fallback
        recommendations_bpr[int(user_id)] = [int(x) for x in top_popular_recipes[:K]]
        continue
    
    user_idx = user_to_idx[user_id]
    
    # Verify the user_idx is valid for the matrix
    if user_idx >= user_item_matrix.shape[0]:
        print(f"Warning: user_idx {user_idx} >= matrix rows {user_item_matrix.shape[0]} for user_id {user_id}")
        recommendations_bpr[int(user_id)] = [int(x) for x in top_popular_recipes[:K]]
        continue
    
    user_seen = users_seen_train.get(user_id, set())
    
    try:
        # Get the user's row from the matrix
        user_row = user_item_matrix[user_idx]
        
        # Verify dimensions before calling recommend
        if user_row.shape[1] != len(item_ids):
            print(f"Warning: user_row shape {user_row.shape} doesn't match items {len(item_ids)}")
            recommendations_bpr[int(user_id)] = [int(x) for x in top_popular_recipes[:K]]
            continue
        
        # Get recommendations from BPR
        recommended_items, scores = model_bpr.recommend(
            user_idx, 
            user_row,
            N=K*3,  # Get more than needed to account for filtering
            filter_already_liked_items=True
        )
        
        # Convert back to original item IDs and filter already seen items
        recs = []
        for item_idx in recommended_items:
            if item_idx in idx_to_item:
                item_id = idx_to_item[item_idx]
                if item_id not in user_seen:
                    recs.append(int(item_id))
                if len(recs) >= K:
                    break
        
        # If we don't have enough recommendations, fill with popular items
        if len(recs) < K:
            for pop_item in top_popular_recipes:
                if pop_item not in user_seen and pop_item not in recs:
                    recs.append(int(pop_item))
                if len(recs) >= K:
                    break
        
        recommendations_bpr[int(user_id)] = recs[:K]
    
    except Exception as e:
        # If there's any error, use popularity fallback
        print(f"Error for user {user_id} (idx={user_idx}): {e}")
        recommendations_bpr[int(user_id)] = [int(x) for x in top_popular_recipes[:K]]

# Save recommendations
with open("bpr_recommendations.json", "w") as f:
    json.dump(recommendations_bpr, f, indent=4)

print(f"\nSaved BPR recommendations to 'bpr_recommendations.json'")
print(f"Total users with recommendations: {len(recommendations_bpr)}")

Total training interactions: 95000
Number of users: 16077
Number of items: 54028
Test users not in training: 363 out of 2834
Sparse matrix shape: (16077, 54028)
Number of interactions: 92121
Sparsity: 0.9999

Training BPR model...


/home/asaladi/miniconda3/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0018491744995117188 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

BPR model trained!


BPR recommendations:   5%|██▊                                                           | 128/2834 [00:00<00:03, 729.70it/s]

...

Error for user 4852 (idx=1473): index 17013 is out of bounds for axis 1 with size 16077
Error for user 3097 (idx=2856): index 25637 is out of bounds for axis 1 with size 16077
Error for user 166 (idx=3202): index 23494 is out of bounds for axis 1 with size 16077
Error for user 9734 (idx=10655): index 34115 is out of bounds for axis 1 with size 16077
Error for user 843 (idx=747): index 42729 is out of bounds for axis 1 with size 16077
Error for user 7852 (idx=11925): index 33013 is out of bounds for axis 1 with size 16077
Error for user 4029 (idx=5334): index 17592 is out of bounds for axis 1 with size 16077
Error for user 4895 (idx=10254): index 24331 is out of bounds for axis 1 with size 16077
Error for user 888 (idx=659): index 19421 is out of bounds for axis 1 with size 16077
Error for user 3874 (idx=12138): index 33975 is out of bounds for axis 1 with size 16077
Error for user 7366 (idx=1940): index 19424 is out of bounds for axis 1 with size 16077
Error for user 7422 (idx=5235): i

BPR recommendations:  91%|██████████████████████████████████████████████████████▊     | 2589/2834 [00:01<00:00, 2191.20it/s]

Error for user 322 (idx=266): index 16826 is out of bounds for axis 1 with size 16077
Error for user 968 (idx=1819): index 22274 is out of bounds for axis 1 with size 16077
Error for user 3072 (idx=2392): index 17198 is out of bounds for axis 1 with size 16077
Error for user 13885 (idx=8333): index 19538 is out of bounds for axis 1 with size 16077
Error for user 4387 (idx=543): index 17537 is out of bounds for axis 1 with size 16077
Error for user 1381 (idx=227): index 17761 is out of bounds for axis 1 with size 16077
Error for user 9852 (idx=7536): index 16865 is out of bounds for axis 1 with size 16077
Error for user 15190 (idx=13053): index 39067 is out of bounds for axis 1 with size 16077
Error for user 13558 (idx=2808): index 16617 is out of bounds for axis 1 with size 16077
Error for user 9462 (idx=12537): index 39212 is out of bounds for axis 1 with size 16077
Error for user 4466 (idx=883): index 23176 is out of bounds for axis 1 with size 16077
Error for user 3333 (idx=4858): i

BPR recommendations: 100%|████████████████████████████████████████████████████████████| 2834/2834 [00:02<00:00, 1387.26it/s]

Error for user 1602 (idx=3127): index 16163 is out of bounds for axis 1 with size 16077
Error for user 2059 (idx=2264): index 21995 is out of bounds for axis 1 with size 16077
Error for user 2516 (idx=1142): index 18922 is out of bounds for axis 1 with size 16077
Error for user 3952 (idx=2275): index 21061 is out of bounds for axis 1 with size 16077
Error for user 15344 (idx=15402): index 23709 is out of bounds for axis 1 with size 16077
Error for user 3293 (idx=12055): index 24724 is out of bounds for axis 1 with size 16077
Error for user 14592 (idx=8506): index 20151 is out of bounds for axis 1 with size 16077
Error for user 15508 (idx=10282): index 29966 is out of bounds for axis 1 with size 16077
Error for user 4860 (idx=4795): index 34167 is out of bounds for axis 1 with size 16077
Error for user 1192 (idx=1848): index 17212 is out of bounds for axis 1 with size 16077
Error for user 293 (idx=294): index 17194 is out of bounds for axis 1 with size 16077
Error for user 3791 (idx=738

In [3]:
# Install implicit library first: pip install implicit

import implicit
from scipy.sparse import coo_matrix
import numpy as np
import json
from tqdm import tqdm
import pandas as pd

In [9]:
import implicit
from scipy.sparse import coo_matrix
import numpy as np
import json
from tqdm import tqdm

# Prepare data for BPR

root = "datasets3/"
train_interactions = pd.read_csv(root+"seen_interactions.csv")
test_interactions = pd.read_csv(root+"hidden_interactions.csv")

# Use ALL training data, but weight high ratings more
# This ensures all users are in the model

print(f"Total training interactions: {len(train_interactions)}")

# Create user and item mappings using ALL training data
user_ids = train_interactions['u'].unique()
item_ids = train_interactions['i'].unique()

user_to_idx = {uid: idx for idx, uid in enumerate(user_ids)}
item_to_idx = {iid: idx for idx, iid in enumerate(item_ids)}
idx_to_item = {idx: iid for iid, idx in item_to_idx.items()}
idx_to_user = {idx: uid for uid, idx in user_to_idx.items()}

print(f"Number of users: {len(user_ids)}")
print(f"Number of items: {len(item_ids)}")

# Create user-item interaction matrix with weighted ratings
# Give more weight to higher ratings
user_indices = train_interactions['u'].map(user_to_idx).values
item_indices = train_interactions['i'].map(item_to_idx).values

# Weight strategy: convert ratings to confidence scores
# 5 -> 2.0, 4 -> 1.0, 3 -> 0.5, 2 -> 0.1, 1 -> 0.0 (or exclude)
def rating_to_weight(rating):
    if rating >= 5:
        return 2.0
    elif rating >= 4:
        return 1.0
    elif rating >= 3:
        return 0.5
    elif rating >= 2:
        return 0.1
    else:
        return 0.0

weights = train_interactions['rating'].apply(rating_to_weight).values

# Filter out zero weights
non_zero_mask = weights > 0
user_indices = user_indices[non_zero_mask]
item_indices = item_indices[non_zero_mask]
weights = weights[non_zero_mask]

user_item_matrix = coo_matrix(
    (weights, (user_indices, item_indices)),
    shape=(len(user_ids), len(item_ids))
).tocsr()

print(f"Sparse matrix shape: {user_item_matrix.shape}")
print(f"Number of interactions: {user_item_matrix.nnz}")
print(f"Sparsity: {1 - (user_item_matrix.nnz / (user_item_matrix.shape[0] * user_item_matrix.shape[1])):.4f}")

# Train BPR model
print("\nTraining BPR model...")
model_bpr = implicit.bpr.BayesianPersonalizedRanking(
    factors=100,
    iterations=50,
    learning_rate=0.01,
    regularization=0.01,
    random_state=42,
    verify_negative_samples=True
)

# BPR expects item-user matrix (transposed)
model_bpr.fit(user_item_matrix.T, show_progress=True)

print("BPR model trained!")

# Build set of items each user has seen
users_seen_train = train_interactions.groupby('u')['i'].agg(set).to_dict()

# Popularity fallback
recipe_rating_sum = train_interactions.groupby('i')['rating'].sum()
recipe_rating_count = train_interactions.groupby('i')['rating'].count()
recipe_avg_rating = recipe_rating_sum / recipe_rating_count
recipe_popularity = recipe_avg_rating * recipe_rating_count
top_popular_recipes = list(recipe_popularity.sort_values(ascending=False).index[:K])

# Generate recommendations for test users
K = 30
test_users = test_interactions['u'].unique()

recommendations_bpr = {}

for user_id in tqdm(test_users, desc="BPR recommendations"):
    if user_id not in user_to_idx:
        # Unknown user: use popularity fallback
        recommendations_bpr[int(user_id)] = [int(x) for x in top_popular_recipes[:K]]
        continue
    
    user_idx = user_to_idx[user_id]
    user_seen = users_seen_train.get(user_id, set())
    
    try:
        # Get recommendations from BPR
        # Note: recommend() expects user index and user's row from the matrix
        recommended_items, scores = model_bpr.recommend(
            user_idx, 
            user_item_matrix[user_idx],
            N=K*3,  # Get more than needed to account for filtering
            filter_already_liked_items=True
        )
        
        # Convert back to original item IDs and filter already seen items
        recs = []
        for item_idx in recommended_items:
            if item_idx in idx_to_item:
                item_id = idx_to_item[item_idx]
                if item_id not in user_seen:
                    recs.append(int(item_id))
                if len(recs) >= K:
                    break
        
        # If we don't have enough recommendations, fill with popular items
        if len(recs) < K:
            for pop_item in top_popular_recipes:
                if pop_item not in user_seen and pop_item not in recs:
                    recs.append(int(pop_item))
                if len(recs) >= K:
                    break
        
        recommendations_bpr[int(user_id)] = recs[:K]
    
    except Exception as e:
        # If there's any error, use popularity fallback
        print(f"Error for user {user_id}: {e}")
        recommendations_bpr[int(user_id)] = [int(x) for x in top_popular_recipes[:K]]

# Save recommendations
with open("bpr_recommendations.json", "w") as f:
    json.dump(recommendations_bpr, f, indent=4)

print(f"\nSaved BPR recommendations to 'bpr_recommendations.json'")
print(f"Total users with recommendations: {len(recommendations_bpr)}")

Total training interactions: 95000
Number of users: 16077
Number of items: 54028
Sparse matrix shape: (16077, 54028)
Number of interactions: 92121
Sparsity: 0.9999

Training BPR model...


/home/asaladi/miniconda3/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0009238719940185547 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

BPR model trained!


BPR recommendations:   0%|                                                                         | 0/2834 [00:00<?, ?it/s]

...

BPR recommendations:  93%|███████████████████████████████████████████████████████▉    | 2642/2834 [00:01<00:00, 2079.51it/s]

Error for user 29: index 16209 is out of bounds for axis 1 with size 16077
Error for user 358: index 16246 is out of bounds for axis 1 with size 16077
Error for user 5630: index 25507 is out of bounds for axis 1 with size 16077
Error for user 1927: index 17698 is out of bounds for axis 1 with size 16077
Error for user 9989: index 19389 is out of bounds for axis 1 with size 16077
Error for user 3349: index 24929 is out of bounds for axis 1 with size 16077
Error for user 19355: index 27084 is out of bounds for axis 1 with size 16077
Error for user 646: index 19036 is out of bounds for axis 1 with size 16077
Error for user 6722: index 20122 is out of bounds for axis 1 with size 16077
Error for user 4949: index 16854 is out of bounds for axis 1 with size 16077
Error for user 11701: index 25943 is out of bounds for axis 1 with size 16077
Error for user 8622: index 23168 is out of bounds for axis 1 with size 16077
Error for user 659: index 27025 is out of bounds for axis 1 with size 16077
Er

BPR recommendations: 100%|████████████████████████████████████████████████████████████| 2834/2834 [00:01<00:00, 2084.12it/s]

Error for user 2682: index 21758 is out of bounds for axis 1 with size 16077
Error for user 8085: index 27516 is out of bounds for axis 1 with size 16077
Error for user 7073: index 16389 is out of bounds for axis 1 with size 16077
Error for user 21766: index 51212 is out of bounds for axis 1 with size 16077
Error for user 8791: index 16122 is out of bounds for axis 1 with size 16077
Error for user 2235: index 29780 is out of bounds for axis 1 with size 16077
Error for user 15635: index 30701 is out of bounds for axis 1 with size 16077
Error for user 13094: index 53043 is out of bounds for axis 1 with size 16077
Error for user 9564: index 25140 is out of bounds for axis 1 with size 16077
Error for user 710: index 16789 is out of bounds for axis 1 with size 16077
Error for user 12259: index 25838 is out of bounds for axis 1 with size 16077
Error for user 1204: index 31711 is out of bounds for axis 1 with size 16077
Error for user 9129: index 25692 is out of bounds for axis 1 with size 16


Saved BPR recommendations to 'bpr_recommendations.json'
Total users with recommendations: 2834


In [5]:
# Prepare data for BPR
# BPR works best with implicit feedback, so we'll treat ratings >= 4 as positive interactions
high_ratings = train_interactions[train_interactions['rating'] >= 4].copy()

print(f"Total training interactions: {len(train_interactions)}")
print(f"High-rating interactions (>=4): {len(high_ratings)}")

# Create user and item mappings
user_ids = high_ratings['u'].unique()
item_ids = high_ratings['i'].unique()

user_to_idx = {uid: idx for idx, uid in enumerate(user_ids)}
item_to_idx = {iid: idx for idx, iid in enumerate(item_ids)}
idx_to_item = {idx: iid for iid, idx in item_to_idx.items()}
idx_to_user = {idx: uid for uid, idx in user_to_idx.items()}

print(f"Number of users: {len(user_ids)}")
print(f"Number of items: {len(item_ids)}")

# Create user-item interaction matrix
user_indices = high_ratings['u'].map(user_to_idx).values
item_indices = high_ratings['i'].map(item_to_idx).values

# Use binary weights (1 for all positive interactions)
# Or use actual ratings as confidence
weights = np.ones(len(user_indices))  # Binary
# weights = high_ratings['rating'].values  # Or use ratings as confidence

user_item_matrix = coo_matrix(
    (weights, (user_indices, item_indices)),
    shape=(len(user_ids), len(item_ids))
).tocsr()

print(f"Sparse matrix shape: {user_item_matrix.shape}")
print(f"Sparsity: {1 - (user_item_matrix.nnz / (user_item_matrix.shape[0] * user_item_matrix.shape[1])):.4f}")

# Train BPR model
print("\nTraining BPR model...")
model_bpr = implicit.bpr.BayesianPersonalizedRanking(
    factors=100,
    iterations=50,
    learning_rate=0.01,
    regularization=0.01,
    random_state=42
)

# BPR expects item-user matrix (transposed)
model_bpr.fit(user_item_matrix.T)

print("BPR model trained!")



Total training interactions: 95000
High-rating interactions (>=4): 87630
Number of users: 15209
Number of items: 50552
Sparse matrix shape: (15209, 50552)
Sparsity: 0.9999

Training BPR model...


/home/asaladi/miniconda3/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0004932880401611328 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

BPR model trained!


In [7]:
K = 30

In [8]:
# Build set of items each user has seen
users_seen_train = train_interactions.groupby('u')['i'].agg(set).to_dict()

# Popularity fallback
recipe_rating_sum = train_interactions.groupby('i')['rating'].sum()
recipe_rating_count = train_interactions.groupby('i')['rating'].count()
recipe_avg_rating = recipe_rating_sum / recipe_rating_count
recipe_popularity = recipe_avg_rating * recipe_rating_count
top_popular_recipes = list(recipe_popularity.sort_values(ascending=False).index[:K])

# Generate recommendations for test users

test_users = test_interactions['u'].unique()

recommendations_bpr = {}

for user_id in tqdm(test_users, desc="BPR recommendations"):
    if user_id not in user_to_idx:
        # Unknown user: use popularity fallback
        recommendations_bpr[int(user_id)] = [int(x) for x in top_popular_recipes[:K]]
        continue
    
    user_idx = user_to_idx[user_id]
    user_seen = users_seen_train.get(user_id, set())
    
    # Get recommendations from BPR
    # Note: recommend() expects user index and user's row from the matrix
    recommended_items, scores = model_bpr.recommend(
        user_idx, 
        user_item_matrix[user_idx],
        N=K*3,  # Get more than needed to account for filtering
        filter_already_liked_items=True
    )
    
    # Convert back to original item IDs and filter already seen items
    recs = []
    for item_idx in recommended_items:
        if item_idx in idx_to_item:
            item_id = idx_to_item[item_idx]
            if item_id not in user_seen:
                recs.append(int(item_id))
            if len(recs) >= K:
                break
    
    # If we don't have enough recommendations, fill with popular items
    if len(recs) < K:
        for pop_item in top_popular_recipes:
            if pop_item not in user_seen and pop_item not in recs:
                recs.append(int(pop_item))
            if len(recs) >= K:
                break
    
    recommendations_bpr[int(user_id)] = recs[:K]

# Save recommendations
with open("bpr_recommendations.json", "w") as f:
    json.dump(recommendations_bpr, f, indent=4)

print(f"\nSaved BPR recommendations to 'bpr_recommendations.json'")
print(f"Total users with recommendations: {len(recommendations_bpr)}")

BPR recommendations:   0%|                                                                         | 0/2834 [00:00<?, ?it/s]


IndexError: index 15393 is out of bounds for axis 1 with size 15209